#### Getting Started With Langchain And Open AI

- Get setup with LangCHain, LangSmith and LangServe
- Use the most basic and common components of LangChain : Prompt templates, Models, and Output parsers.
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

### RAG - Retrieval Augmented Generation
1. Data Sources : PDF, JSON, URLs, Images => Data Ingestion Technique 
2. Data Translation : Converting Huge Data to Text Chunks
3. Embedding : Text to vectors
4. Store the vectors in the VectorStore Database


### Vector Database
1. FAISS
2. ChromaDB
3. AstroDB

## Retrieval Chain
Retrieval Chain is an interface, which is responsible for quering vector store DB.
## Data Ingestion With Documents Loaders
- Loading a data set from a specific source.
- https://python.langchain.com/v0.2/docs/integrations/document_loaders/
### Document loaders
- DocumentLoaders load data into the standard LangChain Document format.
- Each DocumentLoader has its own specific parameters, but they can all be invoked in the same way with the .load method.